Use case: 1 window to "blink" 2 candidates manually with a "hot key."

In [ ]:
import os

from ipywidgets import Button, Label, VBox, HBox
from ipyevents import Event

from astropy.coordinates import SkyCoord

from ginga.misc import Datasrc
from ginga.misc.log import get_logger
from ginga.util import wcsmod
from ginga.util.iohelper import get_fileinfo

from astrowidgets import ImageWidget

In [ ]:
# Reading ASDF in Ginga needs this to be specified early on
wcsmod.use('astropy_ape14')

In [ ]:
logger = get_logger('my viewer', log_stderr=True, log_file=None, level=30)

In [ ]:
class BlinkImageWidget(ImageWidget):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        num_images = 10  # Max images in cache; should user be able to set this?
        self.datasrc = Datasrc.Datasrc(num_images)  # Cache
        self._saved_skycoord = None
        self._saved_zoom_level = None

    def load_jwst_asdf(self, filename):
        if filename in self.datasrc:
            image = self.datasrc[filename]
        else:
            import asdf
            from ginga.AstroImage import AstroImage

            image = AstroImage(logger=self.logger)
            image.load_file(filename, data_key='data')

            self.datasrc[filename] = image

        self._viewer.set_image(image)
        
    def load_fits(self, filename):
        bnch = get_fileinfo(filename)
        
        if filename in self.datasrc:
            image = self.datasrc[filename]
            self._viewer.set_image(image)
        else:
            super().load_fits(bnch.filepath, numhdu=bnch.numhdu)
            self.datasrc[filename] = self._viewer.get_image()

    def _mouse_click_cb(self, viewer, event, data_x, data_y):
        image = viewer.get_image()
        if image is None:  # Nothing to do
            return

        if image.wcs.wcs is not None:
            ra, dec = image.pixtoradec(data_x, data_y)
            self._saved_skycoord = SkyCoord(ra, dec, unit='deg')

        super()._mouse_click_cb(self._viewer, event, data_x, data_y)

In [ ]:
w1 = BlinkImageWidget(logger=logger)
w1.click_center = True

In [ ]:
fits_files = [
    'D:\\PLLIM\\STScI\\ssb\\test_data\\jw42424001001_01101_00001_nrca5_assign_wcs.fits',
    'D:\\PLLIM\\STScI\\ssb\\test_data\\jw42424001001_01101_00002_nrca5_assign_wcs.fits',
    'D:\\PLLIM\\STScI\\ssb\\test_data\\jw42424001001_01101_00003_nrca5_assign_wcs.fits']
n_files = len(fits_files)
i_file = None  # This counter will be updated by button

In [ ]:
label1 = Label('Filename')
button1 = Button(description="Next image")
button2 = Button(description="Set zoom level")

In [ ]:
def load_image(i_file):
    filename = fits_files[i_file]
    label1.value = os.path.basename(filename)
    # w1.load_fits(filename)  # FITS
    w1.load_jwst_asdf(filename)  # JWST ASDF-in-FITS

    # Jump to saved coordinates
    image = w1._viewer.get_image()
    if image.wcs.wcs is not None and isinstance(w1._saved_skycoord, SkyCoord):
        w1.center_on(w1._saved_skycoord)
    if w1._saved_zoom_level is not None:
        w1.zoom_level = w1._saved_zoom_level

d = Event(source=button1, watched_events=['click', 'keydown', 'mouseenter'])

def handle_event(event):
    global i_file
    event_type = event['type']
    go_forward = None

    if event_type == 'keydown':
        event_key = event['key']
        if event_key == 'ArrowRight':
            go_forward = True
        elif event_key == 'ArrowLeft':
            go_forward = False
    elif event_type == 'click':
        go_forward = True

    if go_forward is None:  # Nothing to do
        return

    if i_file is None:
        i_file = 0
    elif go_forward:
        i_file = (i_file + 1) % n_files
    else:
        i_file = (i_file - 1) % n_files

    load_image(i_file)
        
d.on_dom_event(handle_event)

In [ ]:
def on_button2_clicked(b):
    w1._saved_zoom_level = w1.zoom_level

button2.on_click(on_button2_clicked)

In [ ]:
VBox([label1, w1, HBox([button1, button2])])

Click "Next image" to start blinking. Alternately, you can also mouse over "Next image" and use the left/right arrow keys instead.

Use `+` or `-` to zoom to the desired zoom level and click "Set zoom level" to save it.